In [2]:
import pandas as pd
import wandb
from pathlib import Path
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import datetime

from matplotlib.pyplot import MultipleLocator
api = wandb.Api(timeout=19)
mpl.rcParams["axes.unicode_minus"] = False

In [4]:
project_name = "hlsong/Drones_flock_DYN/"
with open("runlist.txt", "r") as f:
    runlist = f.read().splitlines()
    for run in runlist:
        print(f"\"{project_name}{run}\",")


"hlsong/Drones_flock_DYN/"mat"",
"hlsong/Drones_flock_DYN/rsl1gebt",
"hlsong/Drones_flock_DYN/kfnyspet",
"hlsong/Drones_flock_DYN/vp8c8n6l",
"hlsong/Drones_flock_DYN/q3gm5qx9",
"hlsong/Drones_flock_DYN/dycm1fi0",
"hlsong/Drones_flock_DYN/8g51bhf4",
"hlsong/Drones_flock_DYN/"mat_jpr"",
"hlsong/Drones_flock_DYN/vwj2h4wp",
"hlsong/Drones_flock_DYN/snydtcsk",
"hlsong/Drones_flock_DYN/wb2ik8m2",
"hlsong/Drones_flock_DYN/mcuh265e",
"hlsong/Drones_flock_DYN/btv7tjq1",
"hlsong/Drones_flock_DYN/f77jta53",
"hlsong/Drones_flock_DYN/vb3hdss7",
"hlsong/Drones_flock_DYN/233akkdl",
"hlsong/Drones_flock_DYN/z8z9yj0q",
"hlsong/Drones_flock_DYN/ln836qy6",
"hlsong/Drones_flock_DYN/2cr0b8dm",
"hlsong/Drones_flock_DYN/"mat_mask"",
"hlsong/Drones_flock_DYN/dj2fqd3f",
"hlsong/Drones_flock_DYN/jtcmm64q",
"hlsong/Drones_flock_DYN/b6yehsr9",
"hlsong/Drones_flock_DYN/sg7t9bt8",
"hlsong/Drones_flock_DYN/4s8ljars",
"hlsong/Drones_flock_DYN/0repqzlg",
"hlsong/Drones_flock_DYN/iu92iv7u",
"hlsong/Drones_flock_DYN/o3f

### export list

In [16]:
from collections import defaultdict
from pathlib import Path
env_name = "SMAC"
map_name = "mmm"
update = False
project_name = f"hlsong/{env_name}_{map_name}/"
save_path = Path(f"./runlist/{env_name}_{map_name}.txt")

runlist_path = Path(f"./runlist/{env_name}_{map_name}_raw.txt")
if not runlist_path.exists() or update:
    # 更新数据 或者 不存在元数据
    with open("runlist.txt", "r") as fr:
        runlist = fr.read().splitlines()
    runlist_path.parent.mkdir(parents=True, exist_ok=True)
    # 把读到的数据写入到runlist_raw.txt
    with open(f"{str(runlist_path)}", "w") as fw:
        for line in runlist:
                print(line, file=fw)
else:
    with open(f"{str(runlist_path)}", "r") as fr:
        runlist = fr.read().splitlines()

order_key = ['mat_mask', 'mat_jpr', 'mat', 'mappo_maska', 'mappo_jpr', 'mappo']
algo_name = {'mat_mask': "MAT_mar", 'mat_jpr': "MAT_jpr", 'mat': "MAT", 'mappo_maska': "MAPPO_mar", 'mappo_jpr': "MAPPO_jpr", 'mappo': "MAPPO"}
dic = defaultdict(list)
save_path = Path(f"./runlist/{env_name}_{map_name}.txt")
save_path.parent.mkdir(parents=True, exist_ok=True)
with open(f"{str(save_path)}", "w") as fw:
    print(f"\t\t\"{map_name}\": {{", file=fw)
    for run in runlist:
        if "\"" in run:
            algo = run.split("\"")[1]
            continue
        dic[algo].append(run)
    for key in order_key:
        print(f"\t\t\t\"{algo_name[key]}\": [", file=fw)
        for run in dic[key]:
            print(f"\t\t\t\t\"{project_name}{run}\",", file=fw)
        print(f"\t\t\t],", file=fw)
    print("\t\t},", file=fw)


In [ ]:
from collections import defaultdict
from pathlib import Path
env_name = "SMAC"
map_name = "mmm"
update = False
project_name = f"hlsong/{env_name}_{map_name}/"
save_path = Path(f"./runlist/{env_name}_{map_name}.txt")

runlist_path = Path(f"./runlist/{env_name}_{map_name}_raw.txt")
project = f"hlsong/{env_name}_{map_name}"
runs = api.runs(project)
dicc = defaultdict(list)
for run in runs:
    algo = run["algorithm_name"]
    dicc[algo] = run
if not runlist_path.exists() or update:
    # 更新数据 或者 不存在元数据
    with open("runlist.txt", "r") as fr:
        runlist = fr.read().splitlines()
    runlist_path.parent.mkdir(parents=True, exist_ok=True)
    # 把读到的数据写入到runlist_raw.txt
    with open(f"{str(runlist_path)}", "w") as fw:
        for line in runlist:
            print(line, file=fw)
else:
    with open(f"{str(runlist_path)}", "r") as fr:
        runlist = fr.read().splitlines()

order_key = ['mat_mask', 'mat_jpr', 'mat', 'mappo_maska', 'mappo_jpr', 'mappo']
algo_name = {'mat_mask': "MAT_mar", 'mat_jpr': "MAT_jpr", 'mat': "MAT", 'mappo_maska': "MAPPO_mar", 'mappo_jpr': "MAPPO_jpr", 'mappo': "MAPPO"}
dic = defaultdict(list)
save_path = Path(f"./runlist/{env_name}_{map_name}.txt")
save_path.parent.mkdir(parents=True, exist_ok=True)
with open(f"{str(save_path)}", "w") as fw:
    print(f"\t\t\"{map_name}\": {{", file=fw)
    for run in runlist:
        if "\"" in run:
            algo = run.split("\"")[1]
            continue
        dic[algo].append(run)
    for key in order_key:
        print(f"\t\t\t\"{algo_name[key]}\": [", file=fw)
        for run in dic[key]:
            print(f"\t\t\t\t\"{project_name}{run}\",", file=fw)
        print(f"\t\t\t],", file=fw)
    print("\t\t},", file=fw)


### export run name

In [ ]:
project = "hlsong/mujoco_ant_8x1"
# runs = api.runs(project , {"$and": [{"config.algorithm_name": 'mat'},]})
# runs = api.runs(project, {"$and": [{"config.algorithm_name": 'mat'}, ]}, order="-created_at")
runs = api.runs(project, order="-created_at")
runsss = [i for i in runs]
run_list = []
config_list = []
name_list = []
state_list = []
for run in runs:
    config = {k: v for k, v in run.config.items() if not k.startswith('_')}
    config_list.append(config)
    run_list.append('/'.join(run.path))
    name_list.append(run.name)
    state_list.append(run.state)

config_df = pd.DataFrame.from_records(config_list)
name_df = pd.DataFrame({'name': name_list})
run_df = pd.DataFrame({'run': run_list})
state_df = pd.DataFrame({'state': state_list})
all_df = pd.concat([name_df, run_df, state_df, config_df, ], axis=1)
project_config = Path.cwd() / 'data' / f"{project.split('/')[-1]}_configs_{datetime.datetime.now():%m%d-%H%M}.csv"
project_config.parent.mkdir(parents=True, exist_ok=True)
all_df.to_csv(project_config)

### run_name_list

### funtion

In [3]:
from visualize import *
from add_enlist import envlist as ENVLISTa


### define

In [4]:
store = True
save_path = './datatest'
step_lenth = None
smooth = 3
smooth_method = 2
from_wandb = True
save_plot = True
plot_path = './plot_resulttest'

### plot one MAP in one ENV to one IMAGE

In [5]:
env_list = ENVLISTa()
env_name = "StarCraft2"
env = env_list[env_name]
scenarios = [key for key in env.keys()]


['3s_vs_5z']


In [9]:
scenario = scenarios[0]
print(f"env: {env_name}")
if from_wandb:
    df, indicator = get_df_from_wandb(env, scenario, store, save_path, smooth, smooth_method, step_lenth)
else:
    df, indicator = get_df_from_local(env_name, scenario, save_path, smooth, smooth_method, step_lenth)
plot_one_scenario(df, indicator, env_name, scenario, save_plot, plot_path)


env: smac


TypeError: unsupported operand type(s) for /: 'PosixPath' and 'dict'

### plot Multi MAPs in one ENV to Multi IMAGEs

In [14]:
env_name = "drone"
env = env_list[env_name]
scenarios = [key for key in env.keys()]

for scenario in scenarios:
    print(f"env: {env_name}")
    if from_wandb:
        df, indicator = get_df_from_wandb(env, scenario, store, save_path ,smooth, smooth_method, step_lenth)
    else:
        df, indicator = get_df_from_local(env_name, scenario, save_path, smooth, smooth_method, step_lenth)
    plot_one_scenario(df, indicator, env_name, scenario, save_plot, plot_path)

KeyError: 'drone'

### plot Multi MAPs in one ENV to one IMAGEs

In [ ]:
env_name = "drone"
env = env_list[env_name]
scenarios = [key for key in env.keys()]
dfargs = {
    "env_name": env_name,
    "env": env,
    "scenarios": scenarios,
    "store": store,
    "save_path": save_path,
    "step_lenth": step_lenth,
    "smooth": smooth,
    "smooth_method": smooth_method,
    "from_wandb": from_wandb,
}
print(f"env: {env_name}")
nsize = (1, 3)
plot_multi_scenario(env_name, scenarios, nsize, save_plot, plot_path, dfargs)
